In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn import neighbors, datasets, svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import matplotlib
#matplotlib.use('GTKAgg')
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
%matplotlib inline
matplotlib.rcParams.update({'font.size': 18})

pip install xlrd==1.2.0.  (latest version of xlrd only imports .xls 

The KNN classifier works on assigning the probability membership in a specifc class based on the distance to other members in a class.

The KNN algorithm randomly takes 80% of the data for training (X_train) and 20% of the data for testing (X_test). Inorder to make the randomization cosistent every time the model is run, a seed of 14 is chosen. 

After fitting the data into the algorithm, the model predicts the assigned_class of the optimzer inroder to help the user chose the type of optimzer they want to utilize for theor specific needs. i.e type of function, number of parametrs, accuracy and time per trial. 

In [2]:
df = pd.read_excel ('data/All Data combined.xlsx')

<ipython-input-2-36f5b1f1bff7>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel ('data/All Data combined.xlsx')


In [3]:
print(len(df))

1500


In [4]:
X = df[['number of trials','accuracy [calc. max/ actual max]','time per trial [s]']]
y = df['assigned_class']

print(len(y))

1500


In [5]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                        random_state=14) #14 is seed

print(len(y_train), len(y_test))

1050 450


since the data in each dimension of X should be scaled similarly for the model, the below code rescales the features. This minimizes the the bias in which dimensioanlity can drastically favor closseness in arbitary dimensions of X.

In [7]:
#Feature scaling
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [8]:
X_test

array([[ 1.12807   ,  1.1675019 ,  1.04447895],
       [ 0.44115322, -0.23317503, -0.44693361],
       [ 1.58601452,  0.47840222, -0.45067036],
       ...,
       [ 0.21218096, -0.93891383, -0.43515322],
       [ 0.44115322,  0.55631219, -0.42495643],
       [-0.0167913 , -0.90518099, -0.4261909 ]])

In [6]:
K_neighbors = 2

In [7]:
#Define the model: Init K-NN
# We create an instance of Neighbours Classifier and fit it to the training data.

clf = neighbors.KNeighborsClassifier(K_neighbors, weights='distance')

# Here we fit it to the data - it figures out the assigned_classes and 
# which training X points belong to each class in Y 
clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=2, weights='distance')

In [8]:
print(clf.classes_, clf.effective_metric_, clf.n_samples_fit_)

[0 1 2 3 4] euclidean 1050


In [9]:
# Predict the test set results
y_test_predicted = clf.predict(X_test)
y_test_probabilities = clf.predict_proba(X_test)
mean_accuracy = clf.score(X_test, y_test)

print("predicted classes", y_test_predicted)
print("KNN probability of belonging to class",y_test_probabilities)
print("Mean accuracy of prediction", mean_accuracy)


predicted classes [4 3 2 3 3 0 2 1 1 1 4 3 3 4 2 1 0 0 1 1 0 2 4 3 2 0 2 1 2 1 4 2 2 4 4 4 1
 4 2 3 0 3 0 3 3 1 4 2 4 2 3 4 4 4 2 1 1 3 4 4 4 1 2 4 1 0 1 2 4 2 2 4 1 3
 4 0 3 4 3 4 2 4 0 1 0 2 0 4 1 3 4 2 1 0 1 0 2 0 1 3 1 4 1 4 2 3 1 3 2 3 2
 0 0 0 0 1 2 0 0 2 3 1 1 4 2 4 4 3 1 1 0 3 1 1 0 1 4 2 4 1 4 4 0 0 2 3 3 0
 1 3 3 4 1 2 3 2 1 0 1 0 3 4 1 4 4 0 0 0 1 3 3 1 1 1 4 4 4 4 3 4 1 3 4 1 4
 2 2 1 2 1 3 0 3 2 2 0 3 3 3 4 4 3 4 0 2 1 0 0 0 2 1 3 2 4 0 1 3 0 1 3 1 0
 3 1 1 3 0 0 4 1 3 3 1 0 4 2 2 2 3 2 0 3 2 1 4 1 2 4 0 1 1 1 4 2 4 3 4 2 4
 1 4 4 1 4 1 3 4 4 3 1 2 3 3 4 0 0 2 4 4 1 4 0 0 1 2 1 3 0 0 2 2 1 1 0 3 3
 3 1 4 2 3 3 3 4 1 1 1 0 2 1 2 4 2 0 1 2 4 1 1 0 0 4 1 2 3 4 4 0 1 3 2 4 0
 4 4 0 1 2 1 2 0 2 4 2 2 1 0 4 2 0 2 0 2 4 1 0 0 3 0 1 2 1 1 0 2 1 3 1 1 3
 2 4 0 2 4 4 3 1 1 2 2 0 4 4 0 0 4 2 2 2 1 1 0 3 3 1 2 1 1 3 4 3 3 4 1 1 1
 3 3 1 1 3 2 0 2 1 4 2 4 2 0 3 1 3 4 4 1 3 3 1 1 1 1 3 0 2 0 1 0 4 2 2 0 3
 2 3 3 3 2 1]
KNN probability of belonging to class [[0.         0.         0.    

In [10]:
sklearn.metrics.mean_squared_error(y_test, y_test_predicted)

1.1444444444444444

A confusion matrix is a matrix that can be used to measure the performance of a machine learning algorithm. Each row of the matrix represents the instances of an actual assigned_class and each column represents the instances of a predicted class. 

In [11]:
# Evaluate Model
cm = confusion_matrix(y_test, y_test_predicted, labels=[0,1,2,3,4,5])
print(cm)

[[47 13  6 11  0  0]
 [10 45 24 15  0  0]
 [14 28 34 20  0  0]
 [ 7 24 22 37  0  0]
 [ 0  0  0  0 93  0]
 [ 0  0  0  0  0  0]]


In [12]:
def precision(label, confusion_matrix):
    """Precision is the fraction of cases where the algorith correctly predicted assigned_class 
       out of all instances where the algorithm predicted (correctly and incorrectly)"""
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()

def recall(label, confusion_matrix):
    """recall is the fraction of cases where the algorithm correctly predicted out of all of 
       the cases which are labelled as a specific assigned_class"""
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    """calculates the precision for the whole classification problem calculates"""
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    """calculates the recall for the whole classification problem calculates"""
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

In [13]:
print("label precision recall")
for label in range(5):
    print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")


label precision recall
    0     0.603  0.610
    1     0.409  0.479
    2     0.395  0.354
    3     0.446  0.411
    4     1.000  1.000


In [14]:
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 

accuracy(cm)

0.5688888888888889